In [3]:
import copy
import glob
import importlib
import itertools
import math
import operator
import os
import random
import time
from os.path import *
import psutil
import shutil

import keras.layers as layers
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

import cnn_builder as cbuild
import cnn_runner as crun
import config
import ddpg.env as denv
import ddpg.main as qmain
import ddpg.learning_nets as ln
import ddpg.task_nets as tn
import dr_methods as drm
import seg_methods as sm
import niftiutils.helper_fxns as hf
import niftiutils.transforms as tr
import niftiutils.visualization as vis
import voi_methods as vm

%matplotlib inline
np.set_printoptions(3, suppress=True)
#np.random.seed(0)

In [4]:
def memory():
    pid = os.getpid()
    py = psutil.Process(pid)
    gigs = py.memory_info()[0]/2.**30
    print('Memory use:', gigs)
    return gigs

In [5]:
importlib.reload(cbuild)
C = config.Config()
dqn_generator = cbuild._train_gen_ddpg()

In [ ]:
importlib.reload(qmain)
Q = qmain.CRSNet(.9)
Q.train(dqn_generator, verbose=True)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

Episode 0 Replay self.q_buffer 660
4853084','4987127','4673851','	Action [0.829 0.793 0.283 0.449 0.8   0.121 0.709 0.901 0.205 0.291] Reward 0.1
	Action [0.31  0.956 0.694 0.598 0.339 0.557 0.466 0.559 0.692 0.708] Reward 8.5
	Action [0.6   0.572 0.15  0.736 0.203 0.22  0.473 0.348 0.702 0.831] Reward -0.7
	Action [0.112 0.147 0.577 0.75  0.856 0.861 0.64  0.615 0.53  0.726] Reward 23.6
	Action [0.824 0.84  0.438 0.43  0.34  0.405 0.783 0.898 0.577 0.799] Reward 11.3
	Action [0.806 0.684 0.452 0.541 0.964 0.934 0.524 0.576 0.776 0.589] Reward 24.6
	Action [0.531 0.237 0.503 0.289 0.843 0.886 0.742 0.606 0.618 0.157] Reward 6.6
	Action [0.648 0.829 0.615 0.432 0.394 0.852 0.548 0.963 0.179 0.894] Reward 4.0
	Action [0.638 0.81  0.712 0.477 0.167 0.119 0.544 0.566 0.424 0.91 ] Reward 0.6
	Action [0

	Action [0.16  0.327 0.035 0.199 0.416 0.969 0.149 0.794 0.101 0.085] Reward 4.5
	Action [0.031 0.74  0.718 0.033 0.316 0.898 0.261 0.946 0.032 0.193] Reward 0.4
	Action [0.078 0.209 0.462 0.017 0.216 0.421 0.086 0.984 0.171 0.045] Reward 6.5
	Action [0.315 0.925 0.719 0.028 0.087 0.882 0.05  0.982 0.662 0.156] Reward -0.9
	Action [0.348 0.389 0.63  0.264 0.125 0.948 0.062 0.923 0.42  0.195] Reward 5.3
	Action [0.1   0.976 0.172 0.066 0.017 0.96  0.181 0.941 0.02  0.435] Reward -0.3
	Action [0.203 0.656 0.751 0.857 0.007 0.956 0.03  0.972 0.078 0.025] Reward 6.4
Memory use: 2.693572998046875
TOTAL REWARD: 286.0 (50 steps)

Episode 2 Replay self.q_buffer 760
4034641','4547885','	Action [0.037 0.976 0.168 0.049 0.304 0.782 0.262 0.917 0.363 0.517] Reward 5.1
	Action [0.38  0.97  0.556 0.154 0.075 0.966 0.055 0.897 0.121 0.851] Reward 2.0
	Action [0.475 0.715 0.278 0.022 0.148 0.682 0.01  0.986 0.012 0.102] Reward -0.1
	Action [0.035 0.927 0.291 0.582 0.054 0.994 0.143 0.811 0.635 0.141] 

	Action [0.165 0.731 0.575 0.387 0.124 0.952 0.715 0.906 0.236 0.345] Reward 3.0
	Action [0.521 0.913 0.174 0.134 0.027 0.548 0.04  0.738 0.027 0.194] Reward 1.9
	Action [0.152 0.721 0.927 0.051 0.846 0.916 0.179 0.966 0.067 0.238] Reward 0.5
	Action [0.079 0.942 0.676 0.076 0.219 0.928 0.051 0.943 0.018 0.497] Reward 0.1
	Action [0.015 0.983 0.865 0.015 0.361 0.828 0.027 0.843 0.303 0.549] Reward 0.3
	Action [0.296 0.564 0.694 0.325 0.086 0.903 0.123 0.99  0.33  0.041] Reward 0.4
	Action [0.023 0.402 0.794 0.048 0.084 0.816 0.079 0.923 0.013 0.047] Reward 0.3
	Action [0.304 0.797 0.604 0.006 0.033 0.929 0.222 0.971 0.262 0.075] Reward -0.3
	Action [0.179 0.917 0.954 0.069 0.017 0.954 0.006 0.903 0.266 0.094] Reward -0.0
Memory use: 2.7316322326660156
TOTAL REWARD: 16.1 (50 steps)

Episode 4 Replay self.q_buffer 860
	Action [0.737 0.761 0.523 0.045 0.53  0.607 0.038 0.991 0.167 0.543] Reward -10.7
	Action [0.17  0.766 0.7   0.097 0.053 0.902 0.05  0.973 0.08  0.232] Reward -2.7
	Action

	Action [0.039 0.874 0.381 0.101 0.228 0.677 0.021 0.963 0.19  0.052] Reward -1.4
	Action [0.049 0.86  0.777 0.244 0.264 0.574 0.008 0.987 0.002 0.088] Reward -0.2
	Action [0.166 0.968 0.823 0.208 0.367 0.884 0.036 0.982 0.217 0.114] Reward -0.5
	Action [0.22  0.839 0.803 0.059 0.219 0.431 0.036 0.948 0.106 0.025] Reward -0.1
	Action [0.123 0.827 0.796 0.017 0.051 0.91  0.145 0.996 0.233 0.215] Reward -0.2
	Action [0.189 0.909 0.747 0.03  0.012 0.597 0.085 0.936 0.012 0.199] Reward -0.1
	Action [0.387 0.888 0.796 0.063 0.387 0.572 0.132 0.998 0.057 0.341] Reward -1.7
	Action [0.026 0.881 0.923 0.313 0.15  0.813 0.072 0.704 0.007 0.157] Reward 0.2
	Action [0.064 0.883 0.852 0.007 0.157 0.961 0.03  0.761 0.137 0.078] Reward -0.7
	Action [0.125 0.751 0.846 0.278 0.372 0.851 0.091 0.959 0.008 0.216] Reward -0.2
Memory use: 2.8651580810546875
TOTAL REWARD: -7.8 (50 steps)

Episode 6 Replay self.q_buffer 960
	Action [0.066 0.798 0.911 0.577 0.188 0.931 0.032 0.799 0.017 0.054] Reward -1.6
	A

	Action [0.15  0.985 0.906 0.227 0.062 0.86  0.006 0.973 0.216 0.031] Reward 0.6
	Action [0.295 0.785 0.93  0.086 0.057 0.93  0.025 0.927 0.146 0.089] Reward 1.0
	Action [0.174 0.985 0.896 0.217 0.293 0.92  0.167 0.939 0.037 0.039] Reward 0.3
	Action [0.07  0.847 0.688 0.045 0.034 0.789 0.236 0.967 0.159 0.24 ] Reward -0.2
	Action [0.027 0.982 0.941 0.402 0.528 0.523 0.015 0.625 0.225 0.055] Reward 0.2
	Action [0.12  0.827 0.883 0.064 0.056 0.914 0.11  0.965 0.16  0.044] Reward -0.6
	Action [0.083 0.496 0.474 0.072 0.139 0.461 0.044 0.862 0.011 0.039] Reward -1.6
	Action [0.422 0.864 0.483 0.044 0.134 0.904 0.04  0.993 0.149 0.102] Reward -2.2
	Action [0.157 0.529 0.948 0.21  0.024 0.904 0.118 0.936 0.259 0.059] Reward 1.9
	Action [0.085 0.951 0.7   0.074 0.178 0.476 0.018 0.98  0.035 0.085] Reward 0.6
	Action [0.353 0.77  0.974 0.238 0.073 0.903 0.084 0.526 0.105 0.172] Reward 1.2
	Action [0.046 0.939 0.973 0.157 0.053 0.716 0.079 0.877 0.032 0.057] Reward -0.4
Memory use: 2.981838226

	Action [0.229 0.962 0.737 0.417 0.718 0.558 0.115 0.97  0.101 0.841] Reward 0.2
	Action [0.262 0.818 0.776 0.01  0.011 0.903 0.172 0.989 0.044 0.414] Reward 0.1
	Action [0.244 0.927 0.918 0.087 0.25  0.903 0.318 0.924 0.431 0.676] Reward -0.1
	Action [0.04  0.497 0.877 0.152 0.068 0.909 0.096 0.901 0.159 0.079] Reward -0.7
	Action [0.031 0.7   0.987 0.685 0.143 0.982 0.063 0.997 0.162 0.053] Reward -0.2
	Action [0.418 0.771 0.959 0.825 0.008 0.697 0.138 0.98  0.011 0.015] Reward -0.1
	Action [0.061 0.914 0.794 0.385 0.25  0.578 0.049 0.84  0.076 0.3  ] Reward -0.2
	Action [0.19  0.945 0.586 0.277 0.122 0.662 0.137 0.933 0.032 0.038] Reward 0.1
	Action [0.079 0.846 0.867 0.086 0.172 0.807 0.282 0.952 0.023 0.241] Reward -0.3
	Action [0.583 0.606 0.944 0.439 0.32  0.687 0.096 0.897 0.152 0.108] Reward -0.9
	Action [0.177 0.899 0.977 0.301 0.049 0.845 0.163 0.908 0.284 0.099] Reward -0.2
	Action [0.277 0.966 0.855 0.077 0.18  0.778 0.217 0.991 0.04  0.052] Reward -0.1
	Action [0.025 0.84

	Action [0.499 0.352 0.896 0.923 0.091 0.795 0.02  0.984 0.296 0.004] Reward 12.7
	Action [0.59  0.958 0.962 0.153 0.282 0.935 0.046 0.919 0.232 0.283] Reward -1.7
	Action [0.192 0.857 0.947 0.285 0.129 0.991 0.104 0.877 0.01  0.143] Reward -1.1
	Action [0.161 0.569 0.9   0.4   0.064 0.698 0.01  0.979 0.084 0.052] Reward 2.5
	Action [0.256 0.929 0.819 0.578 0.102 0.58  0.044 0.981 0.07  0.094] Reward -1.6
	Action [0.267 0.789 0.923 0.013 0.124 0.951 0.107 0.73  0.012 0.128] Reward -0.1
	Action [0.138 0.96  0.766 0.029 0.005 0.458 0.038 0.959 0.011 0.239] Reward 0.0
	Action [0.065 0.793 0.865 0.045 0.089 0.9   0.014 0.956 0.005 0.056] Reward -0.1
	Action [0.031 0.832 0.63  0.125 0.318 0.91  0.012 0.928 0.025 0.018] Reward -1.0
	Action [0.037 0.953 0.993 0.122 0.029 0.975 0.372 0.972 0.091 0.085] Reward 0.0
	Action [0.145 0.709 0.871 0.017 0.1   0.72  0.038 0.852 0.021 0.274] Reward -0.3
	Action [0.14  0.498 0.785 0.62  0.047 0.673 0.05  0.877 0.186 0.109] Reward 7.1
	Action [0.253 0.959

	Action [0.256 0.868 0.911 0.112 0.066 0.82  0.166 0.987 0.008 0.206] Reward -0.3
	Action [0.062 0.969 0.962 0.083 0.043 0.523 0.028 0.953 0.194 0.056] Reward 0.1
	Action [0.189 0.89  0.967 0.217 0.103 0.875 0.046 0.957 0.011 0.093] Reward 0.0
	Action [0.13  0.866 0.905 0.186 0.087 0.783 0.164 0.968 0.057 0.153] Reward -0.1
	Action [0.272 0.844 0.875 0.076 0.222 0.873 0.013 0.854 0.019 0.253] Reward -0.6
	Action [0.022 0.897 0.296 0.016 0.262 0.977 0.017 0.943 0.011 0.174] Reward 0.1
	Action [0.312 0.604 0.976 0.021 0.747 0.954 0.06  0.633 0.085 0.261] Reward 8.0
	Action [0.014 0.962 0.822 0.381 0.217 0.923 0.049 0.952 0.029 0.274] Reward -0.1
	Action [0.473 0.475 0.745 0.046 0.025 0.997 0.078 0.974 0.16  0.071] Reward 0.1
	Action [0.003 0.695 0.832 0.058 0.345 0.845 0.014 0.745 0.32  0.078] Reward 0.2
	Action [0.145 0.997 0.954 0.227 0.505 0.633 0.126 0.727 0.106 0.064] Reward 0.1
	Action [0.145 0.809 0.93  0.139 0.036 0.965 0.097 0.959 0.018 0.059] Reward 0.2
	Action [0.221 0.82  0.9

In [ ]:
Q.save_models()

In [6]:
shutil.rmtree(C.model_dir)

In [7]:
os.makedirs(C.model_dir)

In [24]:
cropI = np.random.normal(size=(1,*C.dims,3))

crop_true_seg = np.random.normal(size=(1,*C.dims,3))
true_cls = np.random.uniform(size=(1,3))

y_true = np.random.normal(size=(1,*C.dims,3))
y_pred = np.random.normal(size=(1,*C.dims,4))

loss_layer = train_model.layers[-1]

In [25]:
for _ in range(10):
    print(loss_layer.get_loss(0, y_true, y_pred, C.loss_weights[1:]))
    memory();

-0.05015051
Memory use: 2.0087203979492188
-0.050445613
Memory use: 2.0087203979492188
-0.050510425
Memory use: 2.008716583251953
-0.049581293
Memory use: 2.008716583251953
-0.049884304
Memory use: 2.008716583251953
-0.048125546
Memory use: 2.008716583251953
-0.049017023
Memory use: 2.008716583251953
-0.04843785
Memory use: 2.0087203979492188
-0.050015423
Memory use: 2.0087203979492188
-0.049520925
Memory use: 2.0087203979492188


In [ ]:
importlib.reload(tn)
for _ in range(10):
    log_vars = [K.get_value(x)[0] for x in train_model.layers[-1].log_vars]
    loss = np.sum(np.exp(-log_vars[0]) * K.get_value(tn.hetero_cls_loss(y_true, y_pred)) + log_vars[0], -1)
    print(np.mean(loss))
    memory();

In [30]:
A = prediction_model.predict(cropI)

In [ ]:
for _ in range(10):
    train_model.train_on_batch([cropI, crop_true_seg, true_cls], None)
    A=prediction_model.predict(cropI);
    print(A[1])
    memory();

In [ ]:
importlib.reload(qmain)
Q = qmain.CRSNet(1.)

replay_conf = {'size': 10000,
        'learn_start': 100,
        'partition_num': 100,
        'total_step': 10000,
        'batch_size': 4}
BATCH_SIZE = replay_conf["batch_size"]

Q.load_models(replay_conf)

In [ ]:
Q.critic.model.summary()

In [ ]:
Q.actor.model.summary()

In [ ]:
Q.env.pred_model.summary()

In [ ]:
importlib.reload(qmain)
Q = qmain.CRSNet(.5)
img, true_seg, true_cls = next(dqn_generator)
seg, seg_var, cls = Q.run(img)

In [ ]:
vis.draw_slices(img)

In [ ]:
vis.draw_slices(true_seg)

In [ ]:
vis.draw_slices(seg, normalize=[0,.05])

In [ ]:
hist = M.fit_generator(gen, steps_per_epoch=2, epochs=2)

prediction_model = get_prediction_model()
trainable_model = get_trainable_model(prediction_model)
trainable_model.compile(optimizer='adam', loss=None)
assert len(trainable_model.layers[-1].trainable_weights) == 2  # two log_vars, one for each output
assert len(trainable_model.losses) == 1
hist = trainable_model.fit([X, Y1, Y2], nb_epoch=nb_epoch, batch_size=batch_size, verbose=0)

[np.exp(K.get_value(log_var[0]))**0.5 for log_var in trainable_model.layers[-1].log_vars]

In [ ]:
importlib.reload(config)
importlib.reload(hf)
importlib.reload(cbuild)
importlib.reload(crun)
C = config.Config('etiology')
T = config.Hyperparams()
T.get_best_hyperparams()
T.epochs = 30
T.steps_per_epoch = 300

In [ ]:
importlib.reload(cbuild)
importlib.reload(dqna)
#dqn_generator = cbuild._train_gen_dqn([])
agent = dqna.train_dqn(dqn_generator, agent)

In [ ]:
agent.cls_model.fit(cls_generator)

In [ ]:
T.padding = ['same', 'same']
T.pool_sizes = [(2,2,2),(2,2,2)]
T.f = [64,64,64,64,64,64,64]
T.skip_con = True
T.epochs = 20

In [ ]:
drm.dcm2npy_batch(acc_nums=["E100113043"])
#vm.reset_accnum('E105464882')

In [ ]:
vm.plot_check(2, "E102088195");

In [ ]:
vm.xref_dirs_with_excel()

In [ ]:
crun.run_fixed_hyperparams([C], hyperparams=T)#C_list)

In [ ]:
importlib.reload(cbuild)
model = cbuild.build_cnn_hyperparams(T)
model.summary(line_length=120)

In [ ]:
importlib.reload(cbuild)
#Z_reader = ['E103312835_1','12823036_0','12569915_0','E102093118_0','E102782525_0','12799652_0','E100894274_0','12874178_3','E100314676_0','12842070_0','13092836_2','12239783_0','12783467_0','13092966_0','E100962970_0','E100183257_1','E102634440_0','E106182827_0','12582632_0','E100121654_0','E100407633_0','E105310461_0','12788616_0','E101225606_0','12678910_1','E101083458_1','12324408_0','13031955_0','E101415263_0','E103192914_0','12888679_2','E106096969_0','E100192709_1','13112385_1','E100718398_0','12207268_0','E105244287_0','E102095465_0','E102613189_0','12961059_0','11907521_0','E105311123_0','12552705_0','E100610622_0','12975280_0','E105918926_0','E103020139_1','E101069048_1','E105427046_0','13028374_0','E100262351_0','12302576_0','12451831_0','E102929168_0','E100383453_0','E105344747_0','12569826_0','E100168661_0','12530153_0','E104697262_0']
X_test, Y_test, train_generator, num_samples, train_orig, Z = cbuild.get_cnn_data(n=4)#, Z_test_fixed=Z_reader)
Z_test, Z_train_orig = Z
X_train_orig, Y_train_orig = train_orig
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [ ]:
model.save(join(C.model_dir, "model_.hdf5"))

In [ ]:
model.predict(X_train_orig[20:40])

In [ ]:
Y_train_orig[:20]

In [ ]:
hist = model.fit_generator(train_generator, steps_per_epoch=T.steps_per_epoch, epochs=T.epochs, validation_data=[X_test, Y_test])#, callbacks=[T.early_stopping])

In [ ]:
#plot_with_bbox(fn_list[2], cls_mapping[wrong_guesses[2]])
Y_pred = model.predict(X_test)
y_true = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_test])
y_pred = np.array([max(enumerate(x), key=operator.itemgetter(1))[0] for x in Y_pred])

cm = confusion_matrix(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average="weighted")

#save_output(Z_test, y_pred, y_true)

print(accuracy_score(y_true, y_pred))
#y_true_simp, y_pred_simp, _ = cnna.merge_classes(y_true, y_pred)
#print(accuracy_score(y_true_simp, y_pred_simp))